<a href="https://colab.research.google.com/github/siting1206/NLP_HW2/blob/main/NLP_assignment2_1129.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install torch
!pip install pandas nltk
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.5 MB 7.5 MB/s 
     |████████████████████████████████| 182 kB 58.3 MB/s 
     |████████████████████████████████| 7.6 MB 49.4 MB/s 


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import torch
import torch.nn as nn
from torch.utils.data.dataset import Dataset
from torch.utils.data import DataLoader
import torch.nn.functional as F
from torch.nn import CrossEntropyLoss
import math
import numpy as np
import time
import torch, pandas as pd
import nltk
import re
nltk.download('punkt')

from transformers import set_seed
set_seed(123)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


cuda:0


In [4]:
train_df = pd.read_csv('/content/drive/MyDrive/NLP_assignment2/Dataset/train_ret2.txt', sep='\t')
valid_df = pd.read_csv('/content/drive/MyDrive/NLP_assignment2/Dataset/valid_ret2.txt', sep='\t')
train_df[['snippet','question','answer']] = train_df[['snippet','question','answer']].apply(lambda x: x.str.strip('\"'))
valid_df[['snippet','question','answer']] = valid_df[['snippet','question','answer']].apply(lambda x: x.str.strip('\"'))

In [5]:
train_df.head()

,snippet,question,answer
0,"pdf arxiv org may 20 , 2016 last 8 years life ...","last 8 years life , galileo house arrest espou...",copernicus
1,"lebowski sweater replica jun 17 , 2013 history...","last 8 years life , galileo house arrest espou...",copernicus
2,"assignment 13 nov 30 , 2014 2 1912 olympian fo...",2 1912 olympian football star carlisle indian ...,jim thorpe
3,amazon web services put records accepts keywor...,2 1912 olympian football star carlisle indian ...,jim thorpe
4,"weather world records average , yuma receives ...","city yuma state record average 4 , 055 hours s...",arizona


In [6]:
valid_df.head()

,snippet,question,answer
0,free flashcards authors studystack spain 1959 ...,"spain 1959 , wrote dangerous summer , story ri...",hemingway
1,"november 14 , 2008 called altitude american au...","spain 1959 , wrote dangerous summer , story ri...",hemingway
2,lowest highest death valley mt whitney walk ba...,valley 282 feet sea level state lowest point w...,california
3,"california facts , map state symbols enchanted...",valley 282 feet sea level state lowest point w...,california
4,kids net au encyclopedia cash dispenser many a...,"like banks , many grocery stores dispensing ca...",atms


In [7]:
train_df['ans_start'] = [y.index(x) if str(x) in y else -1 for x,y in zip(train_df["answer"], train_df["snippet"])]
train_df['ans_end'] = [x+len(y)-1 if str(y) in z else -1 for x,y,z in zip(train_df["ans_start"], train_df["answer"], train_df["snippet"])]

In [8]:
valid_df['ans_start'] = [y.index(x) if str(x) in y else -1 for x,y in zip(valid_df["answer"], valid_df["snippet"])]
valid_df['ans_end'] = [x+len(y)-1 if str(y) in z else -1 for x,y,z in zip(valid_df["ans_start"], valid_df["answer"], valid_df["snippet"])]

In [9]:
train_df.head()

,snippet,question,answer,ans_start,ans_end
0,"pdf arxiv org may 20 , 2016 last 8 years life ...","last 8 years life , galileo house arrest espou...",copernicus,56,66
1,"lebowski sweater replica jun 17 , 2013 history...","last 8 years life , galileo house arrest espou...",copernicus,112,122
2,"assignment 13 nov 30 , 2014 2 1912 olympian fo...",2 1912 olympian football star carlisle indian ...,jim thorpe,-1,-1
3,amazon web services put records accepts keywor...,2 1912 olympian football star carlisle indian ...,jim thorpe,-1,-1
4,"weather world records average , yuma receives ...","city yuma state record average 4 , 055 hours s...",arizona,-1,-1


In [10]:
valid_df.head()

,snippet,question,answer,ans_start,ans_end
0,free flashcards authors studystack spain 1959 ...,"spain 1959 , wrote dangerous summer , story ri...",hemingway,100,109
1,"november 14 , 2008 called altitude american au...","spain 1959 , wrote dangerous summer , story ri...",hemingway,115,124
2,lowest highest death valley mt whitney walk ba...,valley 282 feet sea level state lowest point w...,california,-1,-1
3,"california facts , map state symbols enchanted...",valley 282 feet sea level state lowest point w...,california,0,10
4,kids net au encyclopedia cash dispenser many a...,"like banks , many grocery stores dispensing ca...",atms,45,49


In [11]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/213k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/436k [00:00<?, ?B/s]

In [12]:
train_data_s = train_df['snippet'].tolist()
valid_data_s = valid_df['snippet'].tolist()

train_data_q = train_df['question'].tolist()
valid_data_q = valid_df['question'].tolist()
# for i in range(0,len(train_data_q)):
#   if type(train_data_q[i]) == float:
#     print(i)

In [13]:
train_encodings = tokenizer(train_data_s, train_data_q, truncation=True, padding=True)
val_encodings = tokenizer(valid_data_s, valid_data_q, truncation=True, padding=True)
train_encodings[0]

Encoding(num_tokens=176, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])

In [14]:
tokenizer.decode(train_encodings['input_ids'][0])
tokenizer.decode(val_encodings['input_ids'][0])

'[CLS] free flashcards authors studystack spain 1959, wrote dangerous summer, story rival bullfighters, hemingway 1884 [SEP] spain 1959, wrote dangerous summer, story rival bullfighters [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]'

In [15]:
train_answer = train_df[['ans_start', 'ans_end']].to_dict('records')
print(len(train_answer[0]))
print(train_answer[0]['ans_start'])
print(train_answer[0]['ans_end'])
valid_answer = valid_df[['ans_start', 'ans_end']].to_dict('records')

2
56
66


In [16]:
def add_token_positions(encodings, answers):
    ans_start, ans_end = [],[]

    for i in range(len(answers)):
        if answers[i]['ans_start'] == -1 and answers[i]['ans_end'] == -1:
          ans_start.append(0)
          ans_end.append(0)
          continue

        else:
          ans_start.append(encodings.char_to_token(i, answers[i]['ans_start']))
          ans_end.append(encodings.char_to_token(i, answers[i]['ans_end'] + 1))


        if ans_start[-1] is None:
            ans_start[-1] = 0
            ans_end[-1] = 0
            continue
        
            
        shift = 1
        while ans_end[-1] is None:
            ans_end[-1] = encodings.char_to_token(i, answers[i]['ans_end'] - shift)
            shift += 1
            
    encodings.update({'ans_start':ans_start, 'ans_end':ans_end})

In [17]:
# Convert char_based_id to token_based_id
# Find the corossponding token id after input being tokenized
add_token_positions(train_encodings, train_answer)
add_token_positions(val_encodings, valid_answer)

In [18]:
train_encodings.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'ans_start', 'ans_end'])

In [19]:
tokenizer.decode(train_encodings['input_ids'][0])

"[CLS] pdf arxiv org may 20, 2016 last 8 years life, galileo copernicus house arrest espousing man's theory winter 1971 72, record [SEP] last 8 years life, galileo house arrest espousing man's theory [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]"

In [20]:
# check the tokenizer
print(train_df['answer'][6])
print(tokenizer.decode(train_encodings['input_ids'][6][45:50]))
print(train_encodings['ans_start'][:20])
print(train_encodings['ans_end'][:20])
print(val_encodings['ans_start'][:20])
print(val_encodings['ans_end'][:20])

john adams 
john adams
[21, 29, 0, 0, 0, 3, 45, 47, 0, 35, 1, 5, 3, 0, 24, 0, 6, 34, 3, 1]
[24, 32, 0, 0, 0, 6, 50, 52, 0, 36, 4, 8, 9, 0, 26, 0, 9, 37, 8, 6]
[22, 26, 0, 1, 14, 29, 29, 11, 36, 51, 9, 23, 37, 1, 6, 0, 43, 0, 17, 23]
[25, 29, 0, 6, 16, 31, 34, 16, 40, 55, 12, 26, 40, 4, 9, 0, 48, 0, 19, 25]


In [21]:
class QADataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __getitem__(self, idx):
        return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}

    def __len__(self):
        return len(self.encodings.input_ids)

In [22]:
train_dataset = QADataset(train_encodings)
valid_dataset = QADataset(val_encodings)

In [23]:
next(iter(train_dataset))

{'input_ids': tensor([  101,   185,  1181,  2087,   170,  1197,  8745,  1964,  8916,  1336,
          1406,   117,  1446,  1314,   129,  1201,  1297,   117, 20003,  4759,
          1186, 16743,  4558, 14191,  1402,  6040, 13936,  5674, 19578,  1299,
           112,   188,  2749,  3701,  2507,  5117,   117,  1647,   102,  1314,
           129,  1201,  1297,   117, 20003,  4759,  1186,  1402,  6040, 13936,
          5674, 19578,  1299,   112,   188,  2749,   102,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,   

## Model

In [24]:
from transformers import BertModel

class myModel(torch.nn.Module):

    def __init__(self):

        super(myModel, self).__init__()

        self.bert = BertModel.from_pretrained('bert-base-cased')
        self.fc = nn.Linear(768, 2)
        

    def forward(self, input_ids, attention_mask, token_type_ids):

        output = self.bert(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids, return_dict=True)
        logits = output[0]
        out = self.fc(logits)

        return out

## Train

In [25]:
from transformers import AdamW
from tqdm import tqdm

# Set GPU / CPU
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(device)
# Put model on device
model = myModel().to(device)

optim = AdamW(model.parameters(), lr=1e-6)

cuda


Downloading:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a f

In [26]:
# Pack data into dataloader by batch
batch_size = 8
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=True)

In [27]:
training_epoch = 1
loss_fct = CrossEntropyLoss()

In [28]:
def evaluate(valid_loader):
    model.eval()
    running_loss = 0.0

    with torch.no_grad():
        loop = tqdm(valid_loader, leave=True)
        for batch_id, batch in enumerate(loop):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            token_type_ids = batch['token_type_ids'].to(device)
            ans_start = batch['ans_start'].to(device)
            ans_end = batch['ans_end'].to(device)

            # model output
            outputs = model(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)

            ans_start_logits, ans_end_logits = torch.split(outputs, 1, 2)

            ans_start_logits = ans_start_logits.squeeze(-1).contiguous()
            ans_end_logits = ans_end_logits.squeeze(-1).contiguous()

            ans_start_loss = loss_fct(ans_start_logits, ans_start)
            ans_end_loss = loss_fct(ans_end_logits, ans_end)

            loss = ans_start_loss + ans_end_loss

            running_loss += loss.item()
            if batch_id % 30 == 0 and batch_id != 0:
                print('Validation Epoch {} Batch {} Loss {:.4f}'.format(
                    batch_id + 1, batch_id, running_loss / 30))
                running_loss = 0.0

        print("evaluate loss: ", loss)

In [29]:
for epoch in range(training_epoch):
    model.train()
    running_loss = 0.0
    torch.save(model.state_dict(), '/bert_model_1129')
    loop = tqdm(train_loader, leave=True)
    for batch_id, batch in enumerate(loop):
        # reset
        optim.zero_grad()


        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        token_type_ids = batch['token_type_ids'].to(device)
        ans_start = batch['ans_start'].to(device)
        ans_end = batch['ans_end'].to(device)


        # model output
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)

        ans_start_logits, ans_end_logits = torch.split(outputs, 1, 2)

        ans_start_logits = ans_start_logits.squeeze(-1).contiguous()
        ans_end_logits = ans_end_logits.squeeze(-1).contiguous()

        ans_start_loss = loss_fct(ans_start_logits, ans_start)
        ans_end_loss = loss_fct(ans_end_logits, ans_end)



        loss = ans_start_loss + ans_end_loss

        # calculate loss
        loss.backward()
        # update parameters
        optim.step()

        running_loss += loss.item()
        if batch_id % 100 == 0 and batch_id != 0:
            print('Epoch {} Batch {} Loss {:.4f}'.format(
                batch_id + 1, batch_id, running_loss / 100))
            running_loss = 0.0

        loop.set_description(f'Epoch {epoch}')
        loop.set_postfix(loss=loss.item())
    evaluate(valid_loader)

Epoch 0:   0%|          | 101/24953 [00:29<1:51:42,  3.71it/s, loss=9.55]

Epoch 101 Batch 100 Loss 10.3084


Epoch 0:   1%|          | 201/24953 [00:56<1:56:16,  3.55it/s, loss=8.81]

Epoch 201 Batch 200 Loss 9.4054


Epoch 0:   1%|          | 301/24953 [01:25<2:02:31,  3.35it/s, loss=8.18]

...

Validation Epoch 3421 Batch 3420 Loss 2.7177


 99%|█████████▉| 3452/3474 [05:53<00:02,  9.81it/s]

Validation Epoch 3451 Batch 3450 Loss 2.6105


100%|██████████| 3474/3474 [05:55<00:00,  9.77it/s]

evaluate loss:  tensor(6.8187, device='cuda:0')


In [30]:
torch.save(model.state_dict(), '/content/drive/MyDrive/NLP_assignment2/bert_model_1129')

In [31]:
model.load_state_dict(torch.load('/content/drive/MyDrive/NLP_assignment2/bert_model'))

KeyboardInterrupt: ignored

## Predict

In [32]:
def predict(test_loader):
    predict_pos = []

    model.eval()

    ans_sub_output = []

    loop = tqdm(test_loader, leave=True)
    for batch_id, batch in enumerate(loop):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        token_type_ids = batch['token_type_ids'].to(device)

        # model output
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
        
        ans_start_logits, ans_end_logits = torch.split(outputs, 1, 2)

        ans_start_logits = ans_start_logits.squeeze(-1).contiguous()
        ans_end_logits = ans_end_logits.squeeze(-1).contiguous()

        ans_start_prdict = torch.argmax(ans_start_logits, 1).cpu().numpy()
        ans_end_prdict = torch.argmax(ans_end_logits, 1).cpu().numpy()

        for i in range(len(input_ids)):
            predict_pos.append((ans_start_prdict[i].item(), ans_end_prdict[i].item()))

            ans_sub = tokenizer.decode(input_ids[i][ans_start_prdict[i]:ans_end_prdict[i]+1])
            
            ans_sub_output.append(ans_sub)
    
    return ans_sub_output, predict_pos

In [34]:
valid_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False)

In [35]:
ans_sub_output, predict_pos = predict(valid_loader)

100%|██████████| 3474/3474 [05:56<00:00,  9.74it/s]


In [36]:
def get_output_post_fn(test, ans_sub_output):
    ans_sub = []
    for i in range(len(test)):

        ans_sub_pred = ans_sub_output[i].split()

        if ans_sub_pred is None:
            ans_sub_pred = []
        ans_sub_error_index = ans_sub_pred.index('[SEP]') if '[SEP]' in ans_sub_pred else -1

        if ans_sub_error_index != -1:
            ans_sub_pred = ans_sub_pred[:ans_sub_error_index]


        ans_sub.append(' '.join(ans_sub_pred))

    return ans_sub

In [37]:
ans_sub = get_output_post_fn(valid_df, ans_sub_output)

In [38]:
valid_df['ans_pred'] = ans_sub

In [39]:
valid_df.head()

,snippet,question,answer,ans_start,ans_end,ans_pred
0,free flashcards authors studystack spain 1959 ...,"spain 1959 , wrote dangerous summer , story ri...",hemingway,100,109,hemingway 1884
1,"november 14 , 2008 called altitude american au...","spain 1959 , wrote dangerous summer , story ri...",hemingway,115,124,"[CLS] november 14, 2008 called altitude americ..."
2,lowest highest death valley mt whitney walk ba...,valley 282 feet sea level state lowest point w...,california,-1,-1,death valley m
3,"california facts , map state symbols enchanted...",valley 282 feet sea level state lowest point w...,california,0,10,
4,kids net au encyclopedia cash dispenser many a...,"like banks , many grocery stores dispensing ca...",atms,45,49,atms also


In [44]:
import re
import collections
import string
  
def normalize_answer(s):
    """Lower text and remove punctuation, articles and extra whitespace."""
  
    def remove_articles(text):
        regex = re.compile(r"\b(a|an|the)\b", re.UNICODE)
        return re.sub(regex, " ", text)
  
    def white_space_fix(text):
        return " ".join(text.split())
  
    def remove_punc(text):
        exclude = set(string.punctuation)
        return "".join(ch for ch in text if ch not in exclude)
  
    def lower(text):
        return text.lower()
  
    return white_space_fix(remove_articles(remove_punc(lower(s))))
  
  
def get_tokens(s):
    if not s:
        return []
    return normalize_answer(s).split()
  
  
def compute_exact(a_gold, a_pred):
    return int(normalize_answer(a_gold) == normalize_answer(a_pred))
  
  
def compute_f1(a_gold, a_pred):
    gold_toks = get_tokens(a_gold)
    pred_toks = get_tokens(a_pred)
    common = collections.Counter(gold_toks) & collections.Counter(pred_toks)
    num_same = sum(common.values())
    if len(gold_toks) == 0 or len(pred_toks) == 0:
        # If either is no-answer, then F1 is 1 if they agree, 0 otherwise
        return int(gold_toks == pred_toks)
    if num_same == 0:
        return 0
    precision = 1.0 * num_same / len(pred_toks)
    recall = 1.0 * num_same / len(gold_toks)
    f1 = (2 * precision * recall) / (precision + recall)
    return f1

In [52]:
f1 = 0
for i in range(valid_df.shape[0]):
  print(valid_df.iloc[i]["answer"], valid_df.iloc[i]['ans_pred'])
  f1 += compute_f1(valid_df.iloc[i]["answer"], valid_df.iloc[i]['ans_pred'])
print(f1)

hemingway  hemingway 1884
hemingway  [CLS] november 14, 2008 called altitude american authors spain 1959, wrote dangerous summer, story rival bullfighters hemingway 1884
california  death valley m
california  
atms  atms also
atms  
walt disney  walt disney mi
walt disney  walt disney got
bucharest  bucharest 2
bucharest  prezi all
alaska  alaska biggest
alaska  som michigan longest
texas  
texas  texas oil
npr  [CLS]
npr  [CLS]
mel blanc  [CLS] daffy duck timeline events timetoast timeline
mel blanc  [CLS]
trestle  [CLS] louisiana statewide rail system plan mit also top surface head rail contacts wheels trestle braced
trestle  trestle braced
l frank baum  [CLS]
l frank baum  [CLS] twinkle chubbins astonishing adventures nature twinkle chubbins astonishing adventures nature fairyland best known writing wonderful wizard oz thirteen start reading twinkle chubbins kindle minute l frank baum,
minnesota  minnesota things
minnesota  minnesota m
cia  cia headquarters
cia  central intelligence

KeyboardInterrupt: ignored